In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data

item_categories = pd.read_csv('Data/item_categories.csv')
items = pd.read_csv('Data/items.csv')
shops = pd.read_csv('Data/shops.csv')
sales = pd.read_csv('Data/sales_train_v2.csv')
submission = pd.read_csv('submission.csv')
test = pd.read_csv('Data/test.csv')

In [3]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
print(test.shape)
print(sales.shape)

(214200, 3)
(2935849, 6)


In [6]:
submission.head()

,ID,item_cnt_month
0,0,1.263679
1,1,3.577624
2,2,1.376003
3,3,0.929417
4,4,0.526681


In [7]:
submission['floor_item_cnt_month'] = submission.item_cnt_month.apply(np.floor)
submission['date_block_num'] = 34
submission.head()

,ID,item_cnt_month,floor_item_cnt_month,date_block_num
0,0,1.263679,1.0,34
1,1,3.577624,3.0,34
2,2,1.376003,1.0,34
3,3,0.929417,0.0,34
4,4,0.526681,0.0,34


In [8]:
out = test.merge(submission)
out.head()

,ID,shop_id,item_id,item_cnt_month,floor_item_cnt_month,date_block_num
0,0,5,5037,1.263679,1.0,34
1,1,5,5320,3.577624,3.0,34
2,2,5,5233,1.376003,1.0,34
3,3,5,5232,0.929417,0.0,34
4,4,5,5268,0.526681,0.0,34


In [9]:
month = sales.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum()
month.head()

shop_id  item_id  date_block_num
0        30       1                 31.0
         31       1                 11.0
         32       0                  6.0
                  1                 10.0
         33       0                  3.0
Name: item_cnt_day, dtype: float64

In [10]:
print(month.shape)
print(out.shape)
print(month[5][5232])

(1609124,)
(214200, 6)
date_block_num
31    1.0
Name: item_cnt_day, dtype: float64


In [11]:
print(month[0][32].index)
print(out.shape)

Int64Index([0, 1], dtype='int64', name=u'date_block_num')
(214200, 6)


length = out.shape[0]
k = length
ID = []
shop_id = []
item_id = []
count = []
dbn = []
for i in range(length):
    val = out.iloc[i]
    if i%1000 == 0:
        print(i)
    try:
        times = month[val.shop_id][val.item_id].index
        for j in times:
            ans = month[val.shop_id][val.item_id][j]
            #out.iloc[k] = [k, val.shop_id, val.item_id, ans, ans, j]
            ID.append(k)
            shop_id.append(val.shop_id)
            item_id.append(val.item_id)
            count.append(ans)
            dbn.append(j)
            k += 1
            #print([k, val.shop_id, val.item_id, ans, ans, j])
    except:
        lst = []

df = pd.DataFrame({'ID':ID, 'shop_id':shop_id, 'item_id':item_id, 'item_cnt_month':count, 'floor_item_cnt_month':count, 'date_block_num':dbn})
df.head()

final = out.append(df, ignore_index=True)
print(final.shape)

final.to_csv('complete.csv', index=False)

In [2]:
final = pd.read_csv('complete.csv')
final.head()

,ID,date_block_num,floor_item_cnt_month,item_cnt_month,item_id,shop_id
0,0,34,1.0,1.263679,5037.0,5.0
1,1,34,3.0,3.577624,5320.0,5.0
2,2,34,1.0,1.376003,5233.0,5.0
3,3,34,0.0,0.929417,5232.0,5.0
4,4,34,0.0,0.526681,5268.0,5.0


In [3]:
print(final.shape)

(814359, 6)


In [15]:
print(final.shop_id.unique())

[ 5.  4.  6.  3.  2.  7. 10. 12. 28. 31. 26. 25. 22. 24. 21. 15. 16. 18.
 14. 19. 42. 50. 49. 53. 52. 47. 48. 57. 58. 59. 55. 56. 36. 37. 35. 38.
 34. 46. 41. 44. 39. 45.]


In [16]:
shops = pd.DataFrame({'shop_id': final.shop_id.unique()})
shops.to_csv('shops.csv', index=False)

In [17]:
locations = pd.read_csv('location.csv')
locations.head()

,city,latitude,longitude,name,state
0,Vijayawada,16.521158,80.637620,Airtel Grocery Shop,Andhra Pradesh 520011
1,Gudur,14.146341,79.851748,Guduru Stores,Andhra Pradesh 524101
2,Visakhapatnam,17.800938,83.209947,Freshville Supermarket,Andhra Pradesh 530051
3,Guntur,16.323975,80.414359,Grocery Stores,Andhra Pradesh 522006
4,Ongole,15.501508,80.049350,Kolluri Mart(K- Mart) (Super Market & Food Court),Andhra Pradesh 523001


In [18]:
print(locations.shape)

(42, 5)


In [19]:
shops = pd.read_csv('shops.csv')
shops.head()

,shop_id,city,latitude,longitude,name,state
0,5,Vijayawada,16.521158,80.637620,Airtel Grocery Shop,Andhra Pradesh 520011
1,4,Gudur,14.146341,79.851748,Guduru Stores,Andhra Pradesh 524101
2,6,Visakhapatnam,17.800938,83.209947,Freshville Supermarket,Andhra Pradesh 530051
3,3,Guntur,16.323975,80.414359,Grocery Stores,Andhra Pradesh 522006
4,2,Ongole,15.501508,80.049350,K- Mart,Andhra Pradesh 523001


In [20]:
print(shops.shape)

(42, 6)


In [22]:
final.item_id.nunique()/4

1275

In [25]:
grp = final.groupby(['shop_id', 'item_id'])['item_cnt_month'].apply(list)
grp.head()

shop_id  item_id
2.0      30.0                    [0.820856256581, 1.0, 1.0, 1.0, 1.0]
         31.0               [0.890183288606, 4.0, 1.0, 1.0, 1.0, 1.0]
         32.0       [0.922609849072, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0,...
         33.0       [0.941618878396, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0,...
         38.0                                        [0.813995545871]
Name: item_cnt_month, dtype: object

In [4]:
final['ceil_item_cnt_month'] = np.ceil(final.item_cnt_month)
final.head()

,ID,date_block_num,floor_item_cnt_month,item_cnt_month,item_id,shop_id,ceil_item_cnt_month
0,0,34,1.0,1.263679,5037.0,5.0,2.0
1,1,34,3.0,3.577624,5320.0,5.0,4.0
2,2,34,1.0,1.376003,5233.0,5.0,2.0
3,3,34,0.0,0.929417,5232.0,5.0,1.0
4,4,34,0.0,0.526681,5268.0,5.0,1.0


In [5]:
final.to_csv('complete.csv', index=False)

In [33]:
lst = []
shop_id = []
item_id = []
for i in grp.index:
    l = np.ceil(grp[i[0]][i[1]])
    x = l[0]
    y = l[len(l)-1]
    lst.append(x-y)
    shop_id.append(i[0])
    item_id.append(i[1])
change = pd.DataFrame({'shop_id': shop_id, 'item_id': item_id, 'change': lst})
change.head()

,change,item_id,shop_id
0,0.0,30.0,2.0
1,0.0,31.0,2.0
2,0.0,32.0,2.0
3,0.0,33.0,2.0
4,0.0,38.0,2.0


In [34]:
print(change.shape)

(214200, 3)


In [37]:
print(change.change.nunique())
print(change.change.unique())
print(change.change.max())
print(change.change.min())

87
[ 0.000e+00  1.000e+00  2.000e+00 -3.000e+00 -1.000e+00 -2.000e+00
  3.000e+00  9.000e+00  8.000e+00  4.000e+00  5.000e+00  7.000e+00
 -7.000e+00 -6.000e+00 -4.000e+00 -1.500e+01 -1.000e+01  1.100e+01
  1.500e+01 -8.000e+00  6.000e+00 -5.000e+00 -7.900e+01 -2.300e+01
  1.000e+01 -9.000e+00 -2.700e+01 -6.400e+01 -2.000e+01 -3.400e+01
 -4.200e+01 -1.200e+01 -1.700e+01 -3.700e+01 -1.300e+01 -1.400e+01
 -1.040e+02 -2.400e+01 -2.234e+03 -4.810e+02 -1.100e+01 -1.800e+01
 -5.100e+01 -4.000e+01 -2.100e+01 -2.600e+01 -3.000e+01 -2.200e+01
 -1.900e+01 -1.470e+02 -4.800e+01 -3.100e+01 -3.900e+01 -4.700e+01
 -1.600e+01 -5.500e+01 -7.400e+01 -1.530e+02 -8.600e+01 -2.900e+01
 -4.410e+02 -8.500e+01 -7.200e+01 -2.100e+02 -4.300e+01 -5.300e+01
 -2.500e+01 -2.800e+01 -3.500e+01 -4.110e+02  1.300e+01 -4.400e+01
 -5.600e+01 -3.200e+01 -6.800e+01 -4.500e+01 -5.700e+01 -6.300e+01
 -1.380e+02 -9.900e+01 -4.270e+02 -1.450e+02  1.200e+01 -5.400e+01
 -4.100e+01 -7.000e+01 -9.100e+01]
15.0
-2234.0


In [38]:
change.to_csv('change.csv', index=False)